In [ ]:
import os
import cv2
import json
from tqdm import tqdm
import mediapipe as mp

In [ ]:
# Mediapipe Pose setup for body and hands
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)


In [ ]:
# Initialize output paths
IMAGE_PATH = '../dataset/augmented_images'  # Path to your dataset
OUTPUT_PATH = '../dataset/keypoints'  # Path to save JSON files

In [ ]:
def create_dir(directory_path):
    """Creates a directory if it does not already exist."""
    os.makedirs(directory_path, exist_ok=True)

In [ ]:
def validate_json(output_path, label):
    """
    Validates the JSON file for the given label and prints the total number of images.
    
    Args:
        output_path (str): Path to the folder containing JSON files.
        label (str): Label of the folder (e.g., 'A', '1').
    """
    json_path = os.path.join(output_path, f"{label}.json")
    
    # Check if the file exists
    if os.path.exists(json_path):
        with open(json_path, 'r') as json_file:
            data = json.load(json_file)
            total_images = len(data)
            print(f"Total images written for label {label}: {total_images}")
    else:
        print(f"JSON file not found for label {label}: {json_path}")

In [ ]:
def extract_body_keypoints(image_path):
    """
    Extracts keypoints for the body from the image.
    
    Args:
        image_path (str): Path to the image.
    
    Returns:
        list: A list of keypoints (33 keypoints for full body pose).
    """
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    keypoints_data = []

    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            # Append each x, y, z for each keypoint (normalized coordinates)
            keypoints_data.append({"x": landmark.x, "y": landmark.y, "z": landmark.z})

    # Ensure we have 33 keypoints (if any keypoint is missing, it is padded with zeros)
    if len(keypoints_data) < 33:
        keypoints_data.extend([{"x": 0.0, "y": 0.0, "z": 0.0}] * (33 - len(keypoints_data)))  # Pad with zeros if less than 33 keypoints

    return keypoints_data

In [ ]:
def extract_hand_keypoints(image_path):
    """
    Extracts keypoints for the hands from the image.
    
    Args:
        image_path (str): Path to the image.
    
    Returns:
        list: A list of keypoints (21 keypoints per hand, up to two hands).
    """
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    keypoints_data = []

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                keypoints_data.append({"x": lm.x, "y": lm.y, "z": lm.z})  # Append each x, y, z for each keypoint

    # Ensure we have 42 keypoints (21 per hand, up to two hands detected)
    if len(keypoints_data) < 42:
        keypoints_data.extend([{"x": 0.0, "y": 0.0, "z": 0.0}] * (42 - len(keypoints_data)))  # Pad with zeros if less than 42 keypoints

    return keypoints_data

In [ ]:
def process_images_with_validation(base_path, output_path):
    """
    Processes images to extract body and hand keypoints, then saves them as JSON files.
    
    Args:
        base_path (str): Path to the base folder containing images.
        output_path (str): Path to the folder to save JSON files.
    """
    create_dir(output_path)

    # Label map for directories (you can customize this if needed)
    labels = list("123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ")

    # Initialize a dictionary to hold data for each label
    label_data = {label: [] for label in labels}

    for label in tqdm(labels, desc="Processing labels"):
        input_folder = os.path.join(base_path, label)

        if not os.path.exists(input_folder):
            print(f"Folder not found: {input_folder}")
            continue

        for filename in os.listdir(input_folder):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(input_folder, filename)

                # Extract keypoints for both body and hands
                body_keypoints = extract_body_keypoints(image_path)
                hand_keypoints = extract_hand_keypoints(image_path)

                # Create the formatted dictionary for the image
                image_data = {
                    "image_name": filename,
                    "keypoints": {
                        "body": body_keypoints,  # Store body keypoints
                        "hands": hand_keypoints   # Store hand keypoints (empty if no hands detected)
                    }
                }

                # Append the data to the label's list
                label_data[label].append(image_data)

        # Save the accumulated data for the label to a JSON file
        json_path = os.path.join(output_path, f"{label}.json")
        with open(json_path, 'w') as json_file:
            json.dump(label_data[label], json_file, indent=4)
        
        # Validate and print total images written
        validate_json(output_path, label)

In [ ]:
if __name__ == "__main__":
    process_images_with_validation(IMAGE_PATH, OUTPUT_PATH)
    print("Keypoint extraction completed.")